In [2]:
import datalabframework as dlf

In [3]:
dlf.project.rootpath()

'/home/jovyan/src'

#### Init Spark

In [4]:
engine = dlf.engines.get('spark')
spark = engine.context()

In [5]:
#print out name and version
'{}:{}'.format(engine.info['context'], spark.sparkSession.version)

'spark:2.3.0'

In [6]:
#EXPORT 
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
from pyspark.sql.functions import col

def string_indexer(features):
    for c in features:
        yield StringIndexer(inputCol=c, outputCol=c+'_N')

def onehot(features):
    #todo: reproducable mapping, 
    #      here the mapping depends on the data provided
    for c in features:
        yield OneHotEncoder(inputCol=c, outputCol=c+'_C')

def vectorize(stringCols, numericDiscreteCols, numericContinuosCols):
    #todo: automatically classify columns
    reg_all_discrete_cols = numericDiscreteCols + [c+'_N' for c in stringCols]
    reg_all_cols = numericContinuosCols + [c+'_C' for c in reg_all_discrete_cols]

    print(reg_all_cols)

    si = list(string_indexer(reg_cat_string_cols))
    oh = list(onehot(reg_cat_indexed_cols))
    ar = [VectorAssembler(inputCols=reg_all_cols, outputCol="features")]
    
    stages=si+oh+ar
    return stages

def featurize(df, idCol, labelCol, stringCols, numericDiscreteCols, numericContinuosCols):

    #set the pipeline
    stages = vectorize(stringCols, numericDiscreteCols, numericContinuosCols)
    pipeline = Pipeline(stages=stages)

    #fit
    model = pipeline.fit(df)

    #transform
    features_df = model.transform(df).select(col(idCol).alias('id'), col(labelCol).alias('label'),'features')
    
    return features_df

### Train set

In [7]:
df = engine.read('.etl.clean.train')
df.show()

+-----------+--------+------+------+------------------+-----+-----+-------+--------+------+
|PassengerId|Survived|Pclass|   Sex|               Age|SibSp|Parch|   Fare|Embarked| Title|
+-----------+--------+------+------+------------------+-----+-----+-------+--------+------+
|          1|       0|     3|  male|              22.0|    1|    0|   7.25|       S|    Mr|
|          2|       1|     1|female|              38.0|    1|    0|71.2833|       C|   Mrs|
|          3|       1|     3|female|              26.0|    0|    0|  7.925|       S|  Miss|
|          4|       1|     1|female|              35.0|    1|    0|   53.1|       S|   Mrs|
|          5|       0|     3|  male|              35.0|    0|    0|   8.05|       S|    Mr|
|          6|       0|     3|  male|29.128677373963985|    0|    0| 8.4583|       Q|    Mr|
|          7|       0|     1|  male|              54.0|    0|    0|51.8625|       S|    Mr|
|          8|       0|     3|  male|               2.0|    3|    1| 21.075|     

In [11]:
engine.write(df,'train', mode='overwrite')

TypeError: 'NoneType' object is not subscriptable